In [32]:
import numpy as np
import pandas as pd
import numpy as np
import os 
import time
from google.cloud import bigquery
import pandas_gbq
import glob

In [33]:
client = bigquery.Client()

In [34]:
query = "select * from `repositoriodedadosgpsp.estrutura_organizacional_carreiras.MUNIC_existencia_plano_carreira_exceto_docentes` order by ano"  

In [35]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')

Downloading: 100%|██████████|


In [37]:
df

,ano,sigla_uf,id_municipio,nome_municipio,plano_carreira_vigente
0,2018,RO,1100023,Ariquemes,Sim
1,2018,RO,1100056,Cerejeiras,Sim
2,2018,RO,1100072,Corumbiara,Sim
3,2018,RO,1100106,Guajará-Mirim,Sim
4,2018,RO,1100114,Jaru,Sim
...,...,...,...,...,...
11135,2021,GO,5222302,Vila Propício,Não
11136,2021,PA,1505908,Porto de Moz,Recusa
11137,2021,SC,4216602,São José,Recusa
11138,2021,MA,2106359,Marajá do Sena,Sem dados


In [38]:
df['ano'].unique()

<IntegerArray>
[2018, 2021]
Length: 2, dtype: Int64

In [39]:
os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\Tabelas para o usuário baixar")

In [31]:
df.to_csv("MUNIC_existencia_plano_carreira_exceto_docentes.csv",index=False, encoding="latin-1", sep=';', decimal=',')

-----------------------------------------------------------

Testes

In [142]:
df = pd.read_csv("ESTADIC_perfil_gestor_habitacao_tipo_orgao.csv", encoding="latin-1", sep=';', decimal=',')
df

,cod_uf,caracterizacao_orgao_gestor,genero,idade,cor_raca,grau_instrucao,ano,sigla_uf,uf,faixa_etaria
0,14,Órgão da administração indireta,Masculino,63.0,Parda,Até Ensino Superior Completo,2020,RR,Roraima,Entre 50-64
1,15,Órgão da administração indireta,Masculino,50.0,Parda,Até Ensino Superior Completo,2020,PA,Pará,Entre 30-49
2,11,Setor subordinado a outra secretaria,Feminino,43.0,Parda,Até Ensino Superior Completo,2020,RO,Rondônia,Entre 30-49
3,12,Setor subordinado a outra secretaria,Masculino,39.0,Parda,Até Ensino Superior Completo,2020,AC,Acre,Entre 30-49
4,23,Setor subordinado a outra secretaria,Masculino,66.0,Parda,Até Ensino Médio,2020,CE,Ceará,Acima de 65
5,28,Setor subordinado a outra secretaria,Masculino,39.0,Parda,Até Doutorado,2020,SE,Sergipe,Entre 30-49
6,51,Setor subordinado a outra secretaria,Masculino,65.0,Parda,Até Ensino Superior Completo,2020,MT,Mato Grosso,Entre 50-64
7,22,Setor subordinado diretamente à chefia do Exec...,Feminino,53.0,Parda,Até Ensino Superior Completo,2020,PI,Piauí,Entre 50-64
8,16,Secretaria estadual em conjunto com outras pol...,Masculino,36.0,Parda,Até Pós Graduação ou Mestrado,2020,AP,Amapá,Entre 30-49
9,53,Secretaria estadual em conjunto com outras pol...,Masculino,41.0,Parda,Até Pós Graduação ou Mestrado,2020,DF,Distrito Federal,Entre 30-49


-----------------------------------------------------------

Automatizando troca de csv

In [45]:
##os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\Tabelas para o usuário baixar")

In [46]:
## baixando os atuais com , e exportando com ";"

In [47]:
## listando todos os itens do diretório
##len(glob.glob('*.csv'))

In [48]:
## Serve apenas na primeira vez . 

##for i in list(range(0,len(glob.glob('*.csv')))):
    
   # df= pd.read_csv(glob.glob('*.csv')[i])
    #df.to_csv(glob.glob('*.csv')[i],index=False, encoding="latin-1", sep=';', decimal=',')